# Answering Business Questions Using SQL

The first thing I'll do is look through the database to see what tables are in it. From there, I'll inspect some of the tables to see how I can use them.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table", "view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql
SELECT *
FROM album
LIMIT 5;

 * sqlite:///chinook.db
Done.


album_id,title,artist_id
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2
4,Let There Be Rock,1
5,Big Ones,3


In [4]:
%%sql
SELECT *
FROM artist
LIMIT 5;

 * sqlite:///chinook.db
Done.


artist_id,name
1,AC/DC
2,Accept
3,Aerosmith
4,Alanis Morissette
5,Alice In Chains


In [5]:
%%sql
SELECT *
FROM customer
LIMIT 5;

 * sqlite:///chinook.db
Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


In [6]:
%%sql
SELECT *
FROM invoice
LIMIT 5;

 * sqlite:///chinook.db
Done.


invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
2,30,2017-01-03 00:00:00,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,9.9
3,40,2017-01-05 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,1.98
4,18,2017-01-06 00:00:00,627 Broadway,New York,NY,USA,10012-2612,7.92
5,27,2017-01-07 00:00:00,1033 N Park Ave,Tucson,AZ,USA,85719,16.83


# Finding Most Popular Genres in USA

The first thing we'll need to do is find out what the most popular genres are by both total track count and by percentage. From there, we'll work backward to only include tracks from customers from the US. In order to find the percentage, we'll need to know the total amount of tracks as well.

In [7]:
%%sql
SELECT COUNT(track_id)
FROM track;

 * sqlite:///chinook.db
Done.


COUNT(track_id)
3503


There are 3503 tracks in this table.

In [8]:
%%sql
SELECT
    g.name genre,
    COUNT(t.track_id) track_count,
    COUNT(t.track_id)/3503. track_pct
FROM genre g
INNER JOIN track t ON t.genre_id = g.genre_id
GROUP BY 1

 * sqlite:///chinook.db
Done.


genre,track_count,track_pct
Alternative,40,0.011418783899514702
Alternative & Punk,332,0.09477590636597202
Blues,81,0.02312303739651727
Bossa Nova,15,0.004282043962318013
Classical,74,0.0211247502141022
Comedy,17,0.004852983157293748
Drama,64,0.018270054239223524
Easy Listening,24,0.006851270339708821
Electronica/Dance,30,0.008564087924636025
Heavy Metal,28,0.00799314872966029


Okay, now we've got the exact chart we need, but this is for customers all over the world. Now we need to find a way to only count US customers.

In [9]:
%%sql
SELECT *
FROM customer
WHERE country = "USA"
LIMIT 10;

 * sqlite:///chinook.db
Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
16,Frank,Harris,Google Inc.,1600 Amphitheatre Parkway,Mountain View,CA,USA,94043-1351,+1 (650) 253-0000,+1 (650) 253-0000,fharris@google.com,4
17,Jack,Smith,Microsoft Corporation,1 Microsoft Way,Redmond,WA,USA,98052-8300,+1 (425) 882-8080,+1 (425) 882-8081,jacksmith@microsoft.com,5
18,Michelle,Brooks,None,627 Broadway,New York,NY,USA,10012-2612,+1 (212) 221-3546,+1 (212) 221-4679,michelleb@aol.com,3
19,Tim,Goyer,Apple Inc.,1 Infinite Loop,Cupertino,CA,USA,95014,+1 (408) 996-1010,+1 (408) 996-1011,tgoyer@apple.com,3
20,Dan,Miller,None,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,+1 (650) 644-3358,None,dmiller@comcast.com,4
21,Kathy,Chase,None,801 W 4th Street,Reno,NV,USA,89503,+1 (775) 223-7665,None,kachase@hotmail.com,5
22,Heather,Leacock,None,120 S Orange Ave,Orlando,FL,USA,32801,+1 (407) 999-7788,None,hleacock@gmail.com,4
23,John,Gordon,None,69 Salem Street,Boston,MA,USA,2113,+1 (617) 522-1333,None,johngordon22@yahoo.com,4
24,Frank,Ralston,None,162 E Superior Street,Chicago,IL,USA,60611,+1 (312) 332-3232,None,fralston@gmail.com,3
25,Victor,Stevens,None,319 N. Frances Street,Madison,WI,USA,53703,+1 (608) 257-0597,None,vstevens@yahoo.com,5


Now that we've got a way of pulling just US customers, we'll use a chain of joins to make sure we count only US customers.

In [10]:
%%sql
WITH
us_customers AS (
                SELECT *
                FROM customer
                WHERE country = "USA"
                ),
us_invoices AS (
                SELECT *
                FROM invoice i
                INNER JOIN us_customers us ON i.customer_id = us.customer_id
                ),
us_invoice_line AS 
                (
                SELECT *
                FROM invoice_line il
                INNER JOIN us_invoices ui ON il.invoice_id = ui.invoice_id
                ),
us_track AS
                (
                SELECT *
                FROM track t
                INNER JOIN us_invoice_line uil ON t.track_id = uil.track_id
                )
SELECT COUNT(track_id) FROM us_track;

 * sqlite:///chinook.db
Done.


COUNT(track_id)
1051


Importantly, we first needed to check how many tracks were purchased by US customers, which we will be able to use going forward.

In [11]:
%%sql
WITH
us_customers AS (
                SELECT *
                FROM customer
                WHERE country = "USA"
                ),
us_invoices AS (
                SELECT *
                FROM invoice i
                INNER JOIN us_customers us ON i.customer_id = us.customer_id
                ),
us_invoice_line AS 
                (
                SELECT *
                FROM invoice_line il
                INNER JOIN us_invoices ui ON il.invoice_id = ui.invoice_id
                ),
us_track AS
                (
                SELECT *
                FROM track t
                INNER JOIN us_invoice_line uil ON t.track_id = uil.track_id
                )
SELECT
    g.name genre,
    COUNT(ut.track_id) track_count,
    COUNT(ut.track_id)/1051. track_pct
FROM genre g
INNER JOIN us_track ut ON ut.genre_id = g.genre_id
GROUP BY 1
ORDER BY 3 DESC;

 * sqlite:///chinook.db
Done.


genre,track_count,track_pct
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Pop,22,0.02093244529019981
Latin,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


So, if our new label has hip-hop, punk, pop, and blues, but can only choose 3 genres to get behind, we recommend punk, blues, and pop from this data. It's also worth mentioning that from this data, not having a rock artist appears to be a missed opportunity.

In [12]:
%%sql
SELECT e.*, SUM(i.total)
FROM employee e
INNER JOIN customer c on e.employee_id = c.support_rep_id
INNER JOIN invoice i on c.customer_id = i.customer_id
GROUP BY e.employee_id;

 * sqlite:///chinook.db
Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email,SUM(i.total)
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com,1731.5100000000039
4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com,1584.0000000000034
5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com,1393.920000000002


We can see that Jane Peacock has the most in sales, followed by Margaret Park, followed by Steve Johnson. However, it is worth noting, that is also the order of their hire dates, from oldest to newest. So it is possible this is not indicative of the quality of the employee, but how long they have been working there.

In [13]:
%%sql
    SELECT country, COUNT(customer_id) customer_count
    FROM customer
    GROUP BY country

 * sqlite:///chinook.db
Done.


country,customer_count
Argentina,1
Australia,1
Austria,1
Belgium,1
Brazil,5
Canada,8
Chile,1
Czech Republic,2
Denmark,1
Finland,1


First thing I wanted to do was just to get customer counts for each country so I could get a sense of the data.

In [14]:
%%sql
WITH
customer_counts as (
    SELECT country, COUNT(customer_id) customer_count
    FROM customer
    GROUP BY country
    )
SELECT CASE
    WHEN customer_count > 1 THEN country
    ELSE "Other"
    END AS country_other,
    SUM(customer_count) customer_count_other
FROM customer_counts
GROUP BY country_other;

 * sqlite:///chinook.db
Done.


country_other,customer_count_other
Brazil,5
Canada,8
Czech Republic,2
France,5
Germany,4
India,2
Other,15
Portugal,2
USA,13
United Kingdom,3


From there, I was able to make a new column that would either be the country's name or other, depending on whether they had more than one customer. I used this column to group by, so that all the others would count together.

In [15]:
%%sql
WITH
customer_counts as (
    SELECT country, COUNT(customer_id) customer_count
    FROM customer
    GROUP BY country
    ),
other_grouped as (
    SELECT 
        CASE
            WHEN customer_count > 1 THEN country
            ELSE "Other"
            END AS country_other,
        SUM(customer_count) customer_count
    FROM customer_counts
    GROUP BY country_other
    )
SELECT
    country_other,
    customer_count
FROM
    (
    SELECT *,
        CASE
            WHEN country_other = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM other_grouped
    )
ORDER BY sort ASC;

 * sqlite:///chinook.db
Done.


country_other,customer_count
Brazil,5
Canada,8
Czech Republic,2
France,5
Germany,4
India,2
Portugal,2
USA,13
United Kingdom,3
Other,15


In [16]:
%%sql
SELECT
        c.country,
        i.customer_id,
        COUNT(i.customer_id) order_count,
        SUM(i.total) total
    FROM customer c
    INNER JOIN invoice i ON c.customer_id = i.customer_id
    GROUP BY 2

 * sqlite:///chinook.db
Done.


country,customer_id,order_count,total
Brazil,1,13,108.89999999999998
Germany,2,11,82.17
Canada,3,9,99.99
Norway,4,9,72.27000000000001
Czech Republic,5,18,144.54000000000002
Czech Republic,6,12,128.7
Austria,7,9,69.3
Belgium,8,7,60.38999999999999
Denmark,9,10,37.61999999999999
Brazil,10,12,60.39


In [17]:
%%sql
WITH
customer_counts as (
    SELECT country, COUNT(customer_id) customer_count
    FROM customer
    GROUP BY country
    ),
country_list AS (
    SELECT 
        country
    FROM customer_counts
    WHERE customer_count>1
),
customer_invoice AS (
    SELECT
        CASE
            WHEN c.country IN country_list THEN c.country
            ELSE "Other"
            END AS country,
        i.customer_id,
        COUNT(i.customer_id) order_count,
        SUM(i.total) total
    FROM customer c
    INNER JOIN invoice i ON c.customer_id = i.customer_id
    GROUP BY 2
    ),
main as (
    SELECT 
        country, 
        COUNT(customer_id) customer_count,
        SUM(total) total_sales,
        SUM(total)/COUNT(customer_id) avg_customer_value,
        SUM(total)/SUM(order_count) avg_order_value
    FROM customer_invoice ci
    GROUP BY country
    )
SELECT
    country, 
    customer_count, 
    ROUND(total_sales, 2) total_sales, 
    ROUND(avg_customer_value, 2) avg_customer_value,
    ROUND(avg_order_value, 2) avg_order_value
FROM
    (
    SELECT *,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM main
    )
ORDER BY sort ASC;

 * sqlite:///chinook.db
Done.


country,customer_count,total_sales,avg_customer_value,avg_order_value
Brazil,5,427.68,85.54,7.01
Canada,8,535.59,66.95,7.05
Czech Republic,2,273.24,136.62,9.11
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
India,2,183.15,91.58,8.72
Portugal,2,185.13,92.57,6.38
USA,13,1040.49,80.04,7.94
United Kingdom,3,245.52,81.84,8.77
Other,15,1094.94,73.0,7.45


We can glean all sorts of useful information from this. We can see that the US and "Other", ie smaller countries, are the countries with the most customers. But we can also see that the Czech Republic spends a lot per customer and per order. The UK and India also spend a lot per order.

# Album vs Individual Song Purchases

In [18]:
%%sql
SELECT * FROM invoice_line
LIMIT 50;

 * sqlite:///chinook.db
Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity
1,1,1158,0.99,1
2,1,1159,0.99,1
3,1,1160,0.99,1
4,1,1161,0.99,1
5,1,1162,0.99,1
6,1,1163,0.99,1
7,1,1164,0.99,1
8,1,1165,0.99,1
9,1,1166,0.99,1
10,1,1167,0.99,1


I wanted to get a sense of what the invoice line data looks like.

I am going to use the Group_Concat command to basically concatinate all track_ids on a single invoice into one list. From there, I will check the list to see if it matches any albums.

In [23]:
%%sql
CREATE VIEW invoice_tracks AS
SELECT invoice_id, GROUP_CONCAT(track_id) tracks_purchased, COUNT(track_id) track_count
FROM invoice_line
GROUP BY 1

 * sqlite:///chinook.db
(sqlite3.OperationalError) table invoice_tracks already exists
[SQL: CREATE VIEW invoice_tracks AS
SELECT invoice_id, GROUP_CONCAT(track_id) tracks_purchased, COUNT(track_id) track_count
FROM invoice_line
GROUP BY 1]
(Background on this error at: http://sqlalche.me/e/14/e3q8)


Since I anticipate using these tables a lot, I am going to create a view so that I can reference them easily, as opposed to using with statements over and over.

In [24]:
%%sql
SELECT * FROM invoice_tracks;

 * sqlite:///chinook.db
Done.


invoice_id,tracks_purchased,track_count
1,"1158,1159,1160,1161,1162,1163,1164,1165,1166,1167,1168,1169,1170,1171,1172,1173",16
2,"3476,482,2701,1641,1119,2324,201,819,392,2651",10
3,"2516,2646",2
4,"3448,2560,3336,829,1872,748,1778,2514",8
5,"1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002",17
6,"1045,30",2
7,"3477,3439,1102,711,1988,3024,834,2596,1343,2644,42",11
8,"2231,2509,81,1758,1455,2946,574,1752,2263,1560",10
9,"206,3456,1041,1279,2098,3399,1983,993,196",9
10,"2663,3290",2


By using Group_Concat, we can see all the songs purchased in each invoice.

In [25]:
%%sql
CREATE VIEW album_tracks AS
SELECT album_id, GROUP_CONCAT(track_id) tracks_featured, COUNT(track_id) track_count
FROM track
GROUP BY 1;

 * sqlite:///chinook.db
(sqlite3.OperationalError) table album_tracks already exists
[SQL: CREATE VIEW album_tracks AS
SELECT album_id, GROUP_CONCAT(track_id) tracks_featured, COUNT(track_id) track_count
FROM track
GROUP BY 1;]
(Background on this error at: http://sqlalche.me/e/14/e3q8)


In [26]:
%%sql
SELECT * FROM album_tracks;

 * sqlite:///chinook.db
Done.


album_id,tracks_featured,track_count
1,"1,6,7,8,9,10,11,12,13,14",10
2,2,1
3,"3,4,5",3
4,"15,16,17,18,19,20,21,22",8
5,"23,24,25,26,27,28,29,30,31,32,33,34,35,36,37",15
6,"38,39,40,41,42,43,44,45,46,47,48,49,50",13
7,"51,52,53,54,55,56,57,58,59,60,61,62",12
8,"63,64,65,66,67,68,69,70,71,72,73,74,75,76",14
9,"77,78,79,80,81,82,83,84",8
10,"85,86,87,88,89,90,91,92,93,94,95,96,97,98",14


This shows all tracks featured on an album. Now I need to check each album to see if it shows up in an invoice in its entirety. I am also going to eliminate any albums that have 3 or less songs.

In [29]:
%%sql
WITH
large_album_tracks AS
    (
    SELECT *
    FROM album_tracks
    WHERE track_count > 3
    )
SELECT lat.album_id, it.*
FROM invoice_tracks it
INNER JOIN large_album_tracks lat ON lat.tracks_featured = it.tracks_purchased;

 * sqlite:///chinook.db
Done.


album_id,invoice_id,tracks_purchased,track_count
1,23,"1,6,7,8,9,10,11,12,13,14",10
1,414,"1,6,7,8,9,10,11,12,13,14",10
4,200,"15,16,17,18,19,20,21,22",8
4,230,"15,16,17,18,19,20,21,22",8
5,142,"23,24,25,26,27,28,29,30,31,32,33,34,35,36,37",15
5,427,"23,24,25,26,27,28,29,30,31,32,33,34,35,36,37",15
6,32,"38,39,40,41,42,43,44,45,46,47,48,49,50",13
6,64,"38,39,40,41,42,43,44,45,46,47,48,49,50",13
7,300,"51,52,53,54,55,56,57,58,59,60,61,62",12
8,532,"63,64,65,66,67,68,69,70,71,72,73,74,75,76",14


Success! We have all invoices that feature solely the album at hand! Now we just need to count them, and them compare that to the total number of songs purchased.

In [33]:
%%sql
WITH
large_album_tracks AS
    (
    SELECT *
    FROM album_tracks
    WHERE track_count > 3
    ),
album_invoice AS
    (
    SELECT lat.album_id, it.*
    FROM invoice_tracks it
    INNER JOIN large_album_tracks lat ON lat.tracks_featured = it.tracks_purchased
    )
SELECT COUNT(album_id)
FROM album_invoice;

 * sqlite:///chinook.db
Done.


COUNT(album_id)
110


Great! We know there are 110 invoices that are an album. Now we just need to know the total amount of invoices to find the percentage.

In [36]:
%%sql
SELECT COUNT(DISTINCT invoice_id)
FROM invoice_line;

 * sqlite:///chinook.db
Done.


COUNT(DISTINCT invoice_id)
614


There are 614 total invoices. That means that if 110 were album purchases, 514 were not. Seems like albums make up a small percentage of the total purchases.